In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

In [ ]:
! pip install wandb

In [ ]:
import wandb
from wandb.keras import WandbCallback
wandb.init(project="d2_part1_training")

wandb: Currently logged in as: vinciwakanda (davinci). Use `wandb login --relogin` to force relogin


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Hackaton/data_exploration.csv")
df.head()

,Unnamed: 0,timestamp,indor1,indor2,indor3,external,flat,building,surface,stagione,ora
0,0,2001-02-20 00:00:00,21.75,21.37,21.33,NaN,0.09,0.67,417.0,1,0
1,1,2021-01-10 01:00:00,21.70,21.34,21.33,NaN,0.09,0.66,417.0,1,1
2,2,2021-01-10 02:00:00,21.69,21.33,21.28,NaN,0.09,0.65,417.0,1,2
3,3,2021-01-10 03:00:00,21.60,21.31,21.23,NaN,0.09,0.65,417.0,1,3
4,4,2021-01-10 04:00:00,21.57,21.22,21.22,NaN,0.09,0.65,417.0,1,4


In [ ]:
df['stagione'] = df['stagione'].astype('category')
df['ora'] = df['ora'].astype('category')

# Creo una colonna per ogni stagione
for season in df['stagione'].cat.categories:
    df[f'stagione_{season}'] = (df['stagione'] == season).astype(int)

# Creo una colonna per ogni ora
for hour in df['ora'].cat.categories:
    df[f'ora_{hour}'] = (df['ora'] == hour).astype(int)

# Rimuovo le colonne originali di stagione e ora
df = df.drop(columns=['stagione', 'ora'])

# Mostro il DataFrame risultante
df.head()

,Unnamed: 0,timestamp,indor1,indor2,indor3,external,flat,building,surface,stagione_1,...,ora_14,ora_15,ora_16,ora_17,ora_18,ora_19,ora_20,ora_21,ora_22,ora_23
0,0,2001-02-20 00:00:00,21.75,21.37,21.33,NaN,0.09,0.67,417.0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,2021-01-10 01:00:00,21.70,21.34,21.33,NaN,0.09,0.66,417.0,1,...,0,0,0,0,0,0,0,0,0,0
2,2,2021-01-10 02:00:00,21.69,21.33,21.28,NaN,0.09,0.65,417.0,1,...,0,0,0,0,0,0,0,0,0,0
3,3,2021-01-10 03:00:00,21.60,21.31,21.23,NaN,0.09,0.65,417.0,1,...,0,0,0,0,0,0,0,0,0,0
4,4,2021-01-10 04:00:00,21.57,21.22,21.22,NaN,0.09,0.65,417.0,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Remove irrelevant columns
df = df.drop(columns=['timestamp'])

# Handle missing values by filling with zeros
df = df.fillna(0)

# Split into features (X) and target (y)
X = df.drop(columns=['flat']).values  # Input features
y = df['flat'].values  # Desired output

# Normalize the features
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define data dimensions
numero_di_istanti_temporali = X_train.shape[0]
numero_di_caratteristiche = X_train.shape[1]

# Reshape the input
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)  # Add a dimension
X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)  # Reshape test data

# Create the RNN model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, return_sequences=True, input_shape=(numero_di_caratteristiche, 1)),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1)
])

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Create lists to store evaluation metrics for each epoch
r2_scores = []
mae_scores = []
mse_scores = []

# Training the model
for epoch in range(50):
    model.fit(X_train, y_train, epochs=1, batch_size=32, validation_split=0.2, verbose=1, callbacks=[WandbCallback()])

    # Evaluate on the test set
    y_pred = model.predict(X_test_reshaped)

    # Calculate evaluation metrics
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    r2_scores.append(r2)
    mae_scores.append(mae)
    mse_scores.append(mse)
    wandb.log({"R2": r2})
    wandb.log({"mae": mae})
    wandb.log({"mse": mse})

    print(f'Epoch {epoch + 1} - R2: {r2:.4f}, MAE: {mae:.4f}, MSE: {mse:.4f}')

# Print the final evaluation metrics
final_loss = model.evaluate(X_test_reshaped, y_test)
print("Final Test loss:", final_loss)

# Print the mean of the evaluation metrics over all epochs
print("Mean R2:", np.mean(r2_scores))
print("Mean MAE:", np.mean(mae_scores))
print("Mean MSE:", np.mean(mse_scores))


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


171/171 [==============================] - ETA: 0s - loss: 9.3168

wandb: Adding directory to artifact (/content/wandb/run-20230805_073016-yghy0oa8/files/model-best)... Done. 0.0s


54/54 [==============================] - 2s 14ms/step
Epoch 1 - R2: 0.0001, MAE: 2.3785, MSE: 8.6463
171/171 [==============================] - ETA: 0s - loss: 8.2940

wandb: Adding directory to artifact (/content/wandb/run-20230805_073016-yghy0oa8/files/model-best)... Done. 0.0s


...

54/54 [==============================] - 1s 14ms/step
Epoch 38 - R2: 0.9779, MAE: 0.2817, MSE: 0.1914
171/171 [==============================] - ETA: 0s - loss: 0.2109

wandb: Adding directory to artifact (/content/wandb/run-20230805_073016-yghy0oa8/files/model-best)... Done. 0.0s


54/54 [==============================] - 1s 14ms/step
Epoch 39 - R2: 0.9811, MAE: 0.2595, MSE: 0.1636
54/54 [==============================] - 1s 14ms/step
Epoch 40 - R2: 0.9770, MAE: 0.2996, MSE: 0.1985
54/54 [==============================] - 1s 14ms/step
Epoch 41 - R2: 0.9806, MAE: 0.2584, MSE: 0.1679
54/54 [==============================] - 1s 15ms/step
Epoch 42 - R2: 0.9788, MAE: 0.2659, MSE: 0.1831
54/54 [==============================] - 1s 15ms/step
Epoch 43 - R2: 0.9766, MAE: 0.2891, MSE: 0.2024
54/54 [==============================] - 1s 19ms/step
Epoch 44 - R2: 0.9736, MAE: 0.3220, MSE: 0.2282
54/54 [==============================] - 1s 16ms/step
Epoch 45 - R2: 0.9647, MAE: 0.3746, MSE: 0.3056
54/54 [==============================] - 1s 15ms/step
Epoch 46 - R2: 0.9795, MAE: 0.2539, MSE: 0.1771
54/54 [==============================] - 1s 15ms/step
Epoch 47 - R2: 0.9804, MAE: 0.2581, MSE: 0.1695
54/54 [==============================] - 1s 15ms/step
Epoch 48 - R2: 0.9805, MAE: 

wandb: Adding directory to artifact (/content/wandb/run-20230805_073016-yghy0oa8/files/model-best)... Done. 0.0s


54/54 [==============================] - 1s 19ms/step
Epoch 50 - R2: 0.9826, MAE: 0.2508, MSE: 0.1509
54/54 [==============================] - 1s 20ms/step - loss: 0.1509
Final Test loss: 0.150861456990242
Mean R2: 0.7809136293782
Mean MAE: 0.8075271546443618
Mean MSE: 1.8945097720693393


In [ ]:
# Specify the directory for exporting the model to SavedModel format
export_dir = '/content/drive/MyDrive/Colab Notebooks/Hackaton/d2part1'

# Export the model to SavedModel format in the specified directory
tf.saved_model.save(model, export_dir)